# Chapter 4 用張量表示現實中的資料

## Session 4.1 圖片資料

* 一張圖片資料可以看作一堆**純量的集合**，純量依序排列：高度、寬度的網格當中（也就是像素pixel）。
* 每一個像素可能帶有**單個純量值**，如灰階值。 
* 每一個像素可能帶有**多個純量值**，如不同顏色RGB值；**特徵值**，如景深鏡頭的深度。
* 單個像素值通常會使用**8位元整數**進行編碼（12 or 16為精度更高） <br>
* **img_arr** 是什麼？
>  img_arr為類似numpy arrary的物件<br>
有3個軸：2空間軸（寬高）、1色彩通道軸
* Pytorch 要求圖片資料：C(色彩通道)H(高度)W(寬度)，只要是處理numpy資料後續轉換都很方便！<br> 惟需要注意：<font color=#FF0000>不同軸的排列順序與Pytorch是否吻合</font><br> **解法：**<font color=blue>permute() 函式</font>

* 對資料進行轉換：normalization, rotate, scaling (ch12.6)

In [8]:
import numpy as np
import torch
import imageio
img_arr = imageio.imread("../F1388_Code/data/p1ch4/image-dog/bobby.jpg")
img_arr.shape

#720, 1280為空間大小
#img_arr 為一個3軸張量

/var/folders/yb/xnfk9z6x34527z3924bcjqcr0000gn/T/ipykernel_3916/588946327.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread("../F1388_Code/data/p1ch4/image-dog/bobby.jpg")


(720, 1280, 3)

In [10]:
img = torch.from_numpy(img_arr) #先將img_arr轉成pytorch tensor 
out = img.permute(2,0,1)
out.shape

torch.Size([3, 720, 1280])

In [12]:
# 由於需要一次輸入多張照片，在第0軸 插入 Batch 批次軸，使tensor變成 NCHW 的4軸張量
batch_size = 100 
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8) #預先建立一個4軸tensor，色彩強度8位元正整數

In [13]:
import os
data_dir = "../F1388_Code/data/p1ch4/image-cats/" #設定圖檔所在資料夾路徑
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == ".png"]

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1) # 調整tensor中各軸排列順序
    img_t = img_t[:3] #有些圖片會有表示透明度的第3軸，是我們不需要的，故只保留前面3軸

    batch[i] = img_t

batch.shape

/var/folders/yb/xnfk9z6x34527z3924bcjqcr0000gn/T/ipykernel_3916/1620507834.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


torch.Size([100, 3, 256, 256])

In [15]:
# 將像素值左normalization
# method 1
batch = batch.float()
batch /= 255.0
# method 2 standarlization

n_channels = batch.shape[1]  #取得色彩通道的數量

for c in range(n_channels): #依序走訪每個色彩通道
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c] = (batch[:,c] - mean)/std

## Session 4.2 3D立體圖片資料

In [4]:
import imageio
import torch
dir_path = "../F1388_Code/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr =imageio.volread(dir_path, 'DICOM')
vol_arr.shape

#(99, 512, 512)通道軸被省略掉了 
#99 張數
#512 512 為尺寸

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [6]:
# Pytorch要求個軸排列順序 N x C x H x W
vol = torch.from_numpy(vol_arr).float() #先把資料轉成浮點數張量
vol = torch.unsqueeze(vol, 1 ) #在第2軸位置增加插入一個通道軸，dim=1
vol = torch.unsqueeze(vol, 2 ) #在第3軸位置增加插入一個深度軸，dim=1
vol.shape

torch.Size([99, 1, 1, 512, 512])